In [1]:
name_map = {
    "llama.v1.0":"TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "llama.v1.1":"TinyLlama/TinyLlama_v1.1",
    "llama.math":"TinyLlama/TinyLlama_v1.1_math_code",
    "v1.0":"JanDkff/TinyFuncCoder-v1.0",
    "v1.1":"JanDkff/TinyFuncCoder-v1.1"
}


In [2]:
#evaluate_finetuned = False

reformatted_prompt = True
reformatted_path = "R" if reformatted_prompt else ""


eval_types=["humaneval","mbpp","mbppplus","multiple-sh","multiple-cpp","multiple-cs","multiple-js","multiple-java","multiple-php","multiple-ts"] #,"humanevalsynthesize-python","humanevalsynthesize-cpp","humanevalsynthesize-js","humanevalsynthesize-java","ds1000-all-completion"]

names = ["v1.0","v1.1","llama.v1.0","llama.v1.1","llama.math"]

eval_types = ["ds1000-all-completion"]

In [3]:
def reformat_problem(problem):
    p = problem["prompt"]
    doc = p.split("<code>")[0]
    func = "<code>"+ "<code>".join(p.split("<code>")[1:])
    prompt = "# <func> # Python\n"
    for line in doc.split("\n"):
        prompt += f"# {line}\n"
    prompt += func
    return prompt

In [ ]:
# GENERATE DS-1000
from transformers import AutoModelForCausalLM, pipeline, AutoTokenizer
from peft import PeftModelForCausalLM
from datasets import load_dataset
import json

data = load_dataset("xlangai/DS-1000",split="test")

for name in names:
    if name.find("llama") >= 0:
        model = AutoModelForCausalLM.from_pretrained(name_map[name])
        tokenizer = AutoTokenizer.from_pretrained(name_map[name])
    else:
        model = AutoModelForCausalLM.from_pretrained(name_map[f"llama.{name}"])
        peft = PeftModelForCausalLM.from_pretrained(model=model,model_id=name_map[name])
        tokenizer = AutoTokenizer.from_pretrained(name_map[f"llama.{name}"])
        model = peft.merge_and_unload()
    generator = pipeline(task="text-generation",model=model,tokenizer=tokenizer, max_new_tokens=512,device="cuda")
    
    new_data = []
    i = 1
    for problem in data:
        #print(problem)
        p = reformat_problem(problem)
        if not i%100:
            print(f'{name}: {i}/1000')
            print(p)
        problem["output"] = generator(p,temperature=0.2,top_p=0.95,top_k=0,num_return_sequences=1,do_sample=True)[0]["generated_text"]
        new_data.append(problem)
        i += 1
    
    with open(f'R_{name}_ds1000-all-completion.jsonl', 'w') as file:
        for entry in new_data:
            json_line = json.dumps(entry)
            file.write(json_line + "\n")
